# Hybrid (Ensamble) Search

Hybrid Search is a technique that combines multiple search methods to improve retrieval performance. Typically, it combines traditional keyword-based search (like BM25) with semantic search (using embeddings). This approach can provide better results than either method alone, especially for queries that have both keyword-specific and semantic aspects.

## The main benefits of Hybrid (Ensamble) Search are:

1. Better handling of exact matches and rare terms (strength of keyword search)
Understanding of semantic meaning and context (strength of embedding-based search)
2. Improved performance across a wider range of query types

# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.2.11 \
     langchain-google-genai==1.0.7 \
     langchain-community==0.2.10 \
     langchain-huggingface==0.0.3 \
     einops==0.8.0 \
     faiss_cpu==1.8.0.post1 \
     rank_bm25==0.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 21.7 MB/s eta 0:00:00


# Import related libraries related to Langchain, HuggingfaceEmbedding

In [ ]:
# Import Libraries
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
import getpass
import os

# Provide Google API Key. You can create Google API key at following link

[Google Gemini-Pro API Creation Link](https://console.cloud.google.com/apis/credentials)

[YouTube Video](https://www.youtube.com/watch?v=ZHX7zxvDfoc)



In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key. You can create Huggingface API key at following lin

[Higgingface API Creation Link](https://huggingface.co/settings/tokens)




In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


In [ ]:
# Helper function for printing docs
def pretty_print_docs(docs):
    # Iterate through each document and format the output
    for i, d in enumerate(docs):
        print(f"{'-' * 50}\nDocument {i + 1}:")
        print(f"Content:\n{d.page_content}\n")
        print("Metadata:")
        for key, value in d.metadata.items():
            print(f"  {key}: {value}")
    print(f"{'-' * 50}")  # Final separator for clarity

# Example usage
# Assuming `docs` is a list of Document objects


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
# Load documents from a web URL
documents = WebBaseLoader("https://github.com/hwchase17/chroma-langchain/blob/master/state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [ ]:
# Create Keyword based retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 10  # Number of documents to retrieve

# Create embedding-based retriever
embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
vectorstore = FAISS.from_documents(texts, embeddings)
embedding_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Create ensemble (hybrid) retriever
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, embedding_retriever],
    weights=[0.5, 0.5]
)

In [ ]:
# Use the hybrid retriever
query = "What did the president say about Ketanji Brown Jackson?"
hybrid_results = hybrid_retriever.invoke(query)
pretty_print_docs(hybrid_results)
# Print results
#for i, doc in enumerate(hybrid_results, 1):
#    print(f"Result {i}:")
#    print(doc.page_content[:200] + "...\n")

In [ ]:
# Import the RetrievalQA chain for question-answering tasks
from langchain.chains import RetrievalQA

# Initialize the language model with specified settings
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.3, safety_settings={
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },)

# Create a RetrievalQA chain using the language model and the compression retriever
chain = RetrievalQA.from_chain_type(llm=llm, retriever=hybrid_retriever)

# Invoke the chain with a specific query to get a response
reponse = chain.invoke(query)

# Print the result of the response
print(reponse["result"])